# Lesson 3: Sampling Strategies & Class Imbalance

**Module 3: Data & Pipeline Engineering**  
**Estimated Time**: 4-5 hours  
**Difficulty**: Intermediate

---

## 🎯 Learning Objectives

By the end of this lesson, you will:

✅ Understand why sampling matters at every stage of the ML lifecycle  
✅ Know all major sampling techniques (non-probability and probability)  
✅ Handle class imbalance using SMOTE, focal loss, and resampling  
✅ Implement sampling techniques in Python  
✅ Answer 5 interview questions on sampling and class imbalance  

---

## 📚 Table of Contents

1. [Why Sampling Matters](#1-why-sampling)
2. [Non-Probability Sampling](#2-non-probability)
3. [Probability Sampling](#3-probability)
4. [Practical Implications](#4-implications)
5. [Class Imbalance Handling](#5-class-imbalance)
6. [Hands-On: Implementing Sampling Techniques](#6-hands-on)
7. [Hands-On: SMOTE for Imbalanced Classification](#7-smote)
8. [Exercises](#8-exercises)
9. [Interview Preparation](#9-interview)

---

## 1. Why Sampling Matters <a id='1-why-sampling'></a>

Sampling is the practice of **selecting a subset of data** from a larger pool. In ML, sampling occurs at **every stage** of the workflow:

| Stage | Sampling Used | Example |
|-------|--------------|----------|
| **Data Collection** | Choosing what real-world data to collect | Selecting which user segments to survey |
| **Labeling** | Selecting a subset for labeling | Choosing 10K images from 1M for annotation |
| **Train/Val/Test Split** | Splitting data into subsets | 80/10/10 split with stratification |
| **Training (SGD)** | Mini-batch selection | Sampling 32 records per batch |
| **Monitoring** | Logging a fraction of predictions | Logging 1% of inference requests |

### The Core Problem

In many cases, we **cannot or do not use all available data**:
- Data is **too large** (training on trillions of records isn’t feasible)
- Obtaining **labels is costly** (so we label a subset)
- We intentionally **down-sample for quicker experimentation**

**Good sampling** can make model development efficient and ensure the model generalizes.

**Poor sampling** can mislead your results — for example, selecting an unrepresentative subset can cause your model to perform well on that subset but **fail in production**.

---

## 2. Non-Probability Sampling <a id='2-non-probability'></a>

Not strictly based on random chance but uses some subjective or practical criteria to pick data.

### 2.1 Convenience Sampling

Select data that is **easiest to obtain**.

**Example:** Using the first 10,000 records from a log because they’re readily at hand.

**Implications:**
- ⚠️ **High risk of bias** — sample may not represent the overall population
- A model trained on data from a single city may not generalize to other regions
- Popular because it’s convenient, but can **skew results significantly**

### 2.2 Snowball Sampling

Use existing sample data to **recruit further data**. Often used in social networks or graphs.

**Example:** You have data on some users, then include their friends, then friends-of-friends.

**Implications:**
- Helpful when you don’t have direct access to data
- ⚠️ Can **over-represent tightly connected communities** and miss isolated samples

### 2.3 Judgment (Purposive) Sampling

Rely on **experts** to hand-pick what data to include.

**Example:** A domain expert selects "important" fraud cases to train on.

**Implications:**
- Can incorporate valuable **domain knowledge**
- ⚠️ Subjective and can reflect **expert’s biases**

### 2.4 Quota Sampling

Ensure certain **predefined quantities** of different sub-groups.

**Example:** Include exactly 100 samples of each class in a classification problem.

**Implications:**
- ✅ Guarantees representation of all groups
- ⚠️ Selection within each group might still be non-random (convenience-based)
- Can introduce bias if the population within each quota is not homogeneous

### When to Use Non-Probability Sampling

Non-probability sampling is often a **starting point** (especially in early prototyping or when data access is limited). However, models built on non-random samples may not be reliable for production.

---

## 3. Probability (Random) Sampling <a id='3-probability'></a>

Every data point has some **probability** of being selected, typically striving for an unbiased sample. Less prone to bias than non-probabilistic methods.

### 3.1 Simple Random Sampling

Each data point has an **equal chance** of being selected. Like shuffling your dataset and picking a subset.

**When it works:** Homogeneous data with no important subgroups.

**⚠️ Problem:** If there are rare but important subgroups, you might pick too few:

> You have transactions with **2% fraud**. A simple random sample of 1,000 might contain ~20 fraud cases. Random fluctuation could give you 5 or 50, **skewing the fraud rate**.

### 3.2 Weighted Sampling

Each sample is given a **weight (probability)** for selection. This allows oversampling certain cases.

```python
import random
# Weighted sampling in Python
items = ['fraud', 'legitimate']
weights = [0.5, 0.5]  # 50/50, even though fraud is only 2% of data
sampled = random.choices(items, weights=weights, k=1000)
```

**Key distinction:**
- **Weighted sampling** → Physically changes dataset composition (more minority samples)
- **Sample weights in training** → Includes all data but gives more importance in the loss function

Both aim to handle imbalance, but weighted sampling changes the dataset while sample weights change the optimization.

### 3.3 Stratified Sampling

Divide the population into **strata (groups)** and sample from each group separately to ensure representation.

**Example:** For an imbalanced classification, stratify by class label so your train/test split has the **same class proportions** as the full dataset.

```python
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
```

**✅ Advantage:** Greatly reduces variance between subgroup representation.

**⚠️ Drawback:** Need to know the important grouping variables upfront.

### 3.4 Reservoir Sampling

An algorithm for sampling from **streaming data of unknown size**, ensuring each item has an equal probability.

**Use case:** Maintain a random sample of fixed size from a continuous stream (e.g., user clicks) without storing it all.

```python
import random

def reservoir_sampling(stream, k):
    """Maintain a random sample of size k from a stream."""
    reservoir = []
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            # Replace element at random position with decreasing probability
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir
```

This is crucial in **production streaming pipelines** where you can’t store all data.

### 3.5 Importance Sampling

A more advanced technique used in **statistical estimation and reinforcement learning**.

**Idea:** Sample from a different distribution than the target, and correct for the difference using importance weights.

**Example in RL:** Sample episodes from a behavior policy but evaluate a target policy. Importance sampling provides a way to correct for the difference.

---

## 4. Practical Implications of Sampling Choices <a id='4-implications'></a>

| Implication | Description |
|------------|-------------|
| **Bias** | Non-representative samples lead to biased models |
| **Variance** | Small samples have high variance in estimates |
| **Generalization** | Training sample must match production distribution |
| **Evaluation** | Test set must be representative for valid metrics |
| **Temporal** | For time-series, sample order matters (no future data in training!) |

### Golden Rules

1. **Always use stratified sampling** for classification train/test splits
2. **Never shuffle time-series data** — use temporal splits
3. **Document your sampling strategy** — it affects reproducibility
4. **Monitor for sampling bias** — compare sample statistics to population

---

## 5. Class Imbalance Handling <a id='5-class-imbalance'></a>

Class imbalance is one of the most common challenges in real-world ML. Most production datasets are heavily imbalanced.

### 5.1 The Problem

| Domain | Positive Rate | Challenge |
|--------|--------------|------------|
| Fraud detection | 0.1% - 2% | Missing fraud costs millions |
| Medical diagnosis | 1% - 5% | Missing disease can be fatal |
| Click prediction | 1% - 5% | 95% of predictions are "no click" |
| Spam detection | 10% - 20% | Relatively manageable |

A naive model can achieve 99.9% accuracy on fraud data by **always predicting legitimate** — but catches zero fraud!

### 5.2 Data-Level Approaches

#### Oversampling (Minority Class)
- **Random Oversampling**: Duplicate minority samples randomly
- **SMOTE**: Generate synthetic minority samples by interpolating between existing ones
- **ADASYN**: Like SMOTE but focuses on harder-to-learn samples

#### Undersampling (Majority Class)
- **Random Undersampling**: Remove majority samples randomly
- **Tomek Links**: Remove majority samples that are close to minority samples
- **NearMiss**: Select majority samples closest to minority samples

#### Combined Approaches
- **SMOTE + Tomek Links**: Oversample minority, then clean boundary
- **SMOTE + Edited Nearest Neighbors (ENN)**: Oversample + clean noisy samples

### 5.3 Algorithm-Level Approaches

#### Class Weights
Give more importance to minority class in the loss function:
```python
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight='balanced')
```

#### Focal Loss
A modified cross-entropy loss that **down-weights easy examples** and focuses on hard ones:
```
FL(p_t) = -α_t * (1 - p_t)^γ * log(p_t)
```
- When γ = 0, focal loss = standard cross-entropy
- When γ > 0, easy examples are down-weighted
- Originally proposed for object detection (RetinaNet)

#### Ensemble Methods
- **BalancedRandomForest**: Random forest with undersampling per tree
- **EasyEnsemble**: Multiple undersampled subsets, ensemble their predictions

---

## 6. Hands-On: Implementing Sampling Techniques <a id='6-hands-on'></a>

In [ ]:
import numpy as np
import pandas as pd
import random
from collections import Counter

np.random.seed(42)
random.seed(42)

# ============================================================
# Create a synthetic population dataset
# ============================================================
n = 100000
population = pd.DataFrame({
    'user_id': range(n),
    'region': np.random.choice(['US', 'EU', 'Asia', 'LatAm', 'Africa'], 
                               n, p=[0.3, 0.25, 0.25, 0.1, 0.1]),
    'age': np.random.normal(35, 12, n).astype(int).clip(18, 80),
    'purchase_amount': np.random.exponential(50, n).round(2),
    'is_fraud': np.random.choice([0, 1], n, p=[0.98, 0.02])  # 2% fraud
})

print(f"Population size: {len(population):,}")
print(f"\nRegion distribution:")
print(population['region'].value_counts(normalize=True).round(3))
print(f"\nFraud rate: {population['is_fraud'].mean():.4f} ({population['is_fraud'].sum()} cases)")

In [ ]:
# ============================================================
# 1. Simple Random Sampling
# ============================================================
sample_size = 1000

simple_sample = population.sample(n=sample_size, random_state=42)

print("SIMPLE RANDOM SAMPLING")
print(f"Sample size: {len(simple_sample)}")
print(f"\nRegion distribution (sample vs population):")
comparison = pd.DataFrame({
    'Population': population['region'].value_counts(normalize=True).round(3),
    'Sample': simple_sample['region'].value_counts(normalize=True).round(3)
})
print(comparison)
print(f"\nFraud rate - Population: {population['is_fraud'].mean():.4f}")
print(f"Fraud rate - Sample:     {simple_sample['is_fraud'].mean():.4f}")
print(f"Fraud count in sample:   {simple_sample['is_fraud'].sum()} (expected ~{sample_size * 0.02:.0f})")

In [ ]:
# ============================================================
# 2. Stratified Sampling
# ============================================================
from sklearn.model_selection import train_test_split

# Stratified by region
stratified_sample = population.groupby('region', group_keys=False).apply(
    lambda x: x.sample(n=min(200, len(x)), random_state=42)
)

print("STRATIFIED SAMPLING (by region)")
print(f"Sample size: {len(stratified_sample)}")
print(f"\nRegion distribution (sample vs population):")
comparison = pd.DataFrame({
    'Population': population['region'].value_counts(normalize=True).round(3),
    'Stratified': stratified_sample['region'].value_counts(normalize=True).round(3)
})
print(comparison)
print("\n>>> Notice each region is now equally represented (200 each)")

# Stratified train/test split (preserving class proportions)
print("\n--- Stratified Train/Test Split (preserving fraud rate) ---")
X = population.drop('is_fraud', axis=1)
y = population['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(f"Train fraud rate: {y_train.mean():.4f}")
print(f"Test fraud rate:  {y_test.mean():.4f}")
print(f"Population:       {y.mean():.4f}")
print(">>> Proportions are preserved!")

In [ ]:
# ============================================================
# 3. Weighted Sampling
# ============================================================

# Give fraud cases 50x more weight
weights = population['is_fraud'].map({0: 1, 1: 50})
weighted_sample = population.sample(n=1000, weights=weights, random_state=42)

print("WEIGHTED SAMPLING (fraud upweighted 50x)")
print(f"Sample size: {len(weighted_sample)}")
print(f"Fraud rate in sample: {weighted_sample['is_fraud'].mean():.4f}")
print(f"Fraud cases in sample: {weighted_sample['is_fraud'].sum()}")
print(f"\n>>> Population fraud rate was 0.02, now sample has ~{weighted_sample['is_fraud'].mean():.2f}")
print(">>> This enriches rare events for model training")

In [ ]:
# ============================================================
# 4. Reservoir Sampling (streaming)
# ============================================================

def reservoir_sampling(stream, k):
    """Sample k items from a stream of unknown length.
    Each item has equal probability of being in the final sample."""
    reservoir = []
    for i, item in enumerate(stream):
        if i < k:
            reservoir.append(item)
        else:
            j = random.randint(0, i)
            if j < k:
                reservoir[j] = item
    return reservoir

# Simulate a data stream
stream = list(range(100000))  # Simulated stream of 100K events
sample = reservoir_sampling(stream, k=100)

print("RESERVOIR SAMPLING")
print(f"Stream size: {len(stream):,}")
print(f"Reservoir size: {len(sample)}")
print(f"Sample mean: {np.mean(sample):.0f} (expected ~{np.mean(stream):.0f})")
print(f"Sample std:  {np.std(sample):.0f} (expected ~{np.std(stream):.0f})")
print(f"\n>>> Reservoir sampling works without knowing stream size in advance!")
print(">>> Crucial for production streaming pipelines.")

## 7. Hands-On: SMOTE for Imbalanced Classification <a id='7-smote'></a>

SMOTE (Synthetic Minority Oversampling Technique) generates **new synthetic minority samples** by interpolating between existing minority examples.

In [ ]:
# Install if needed: pip install imbalanced-learn
# !pip install imbalanced-learn

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Prepare data
df = population.copy()
le = LabelEncoder()
df['region_encoded'] = le.fit_transform(df['region'])

features = ['age', 'purchase_amount', 'region_encoded']
X = df[features]
y = df['is_fraud']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Training set class distribution:")
print(f"  Legitimate: {(y_train == 0).sum():,}")
print(f"  Fraud:      {(y_train == 1).sum():,}")
print(f"  Ratio:      {(y_train == 0).sum() / (y_train == 1).sum():.0f}:1")

# --- Baseline: No handling ---
print("\n" + "="*60)
print("BASELINE (no imbalance handling)")
print("="*60)
rf_baseline = RandomForestClassifier(n_estimators=100, random_state=42)
rf_baseline.fit(X_train, y_train)
y_pred_baseline = rf_baseline.predict(X_test)
print(classification_report(y_test, y_pred_baseline, target_names=['Legitimate', 'Fraud']))

In [ ]:
# --- Method 1: Class Weights ---
print("="*60)
print("METHOD 1: Class Weights (class_weight='balanced')")
print("="*60)
rf_weighted = RandomForestClassifier(
    n_estimators=100, class_weight='balanced', random_state=42
)
rf_weighted.fit(X_train, y_train)
y_pred_weighted = rf_weighted.predict(X_test)
print(classification_report(y_test, y_pred_weighted, target_names=['Legitimate', 'Fraud']))

In [ ]:
# --- Method 2: SMOTE ---
try:
    from imblearn.over_sampling import SMOTE
    from imblearn.combine import SMOTETomek
    
    print("="*60)
    print("METHOD 2: SMOTE (Synthetic Minority Oversampling)")
    print("="*60)
    
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    
    print(f"Before SMOTE: {Counter(y_train)}")
    print(f"After SMOTE:  {Counter(y_train_smote)}")
    
    rf_smote = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_smote.fit(X_train_smote, y_train_smote)
    y_pred_smote = rf_smote.predict(X_test)
    print(classification_report(y_test, y_pred_smote, target_names=['Legitimate', 'Fraud']))
    
    # --- Method 3: SMOTE + Tomek Links ---
    print("="*60)
    print("METHOD 3: SMOTE + Tomek Links (Combined)")
    print("="*60)
    
    smt = SMOTETomek(random_state=42)
    X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)
    
    print(f"After SMOTE+Tomek: {Counter(y_train_smt)}")
    
    rf_smt = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_smt.fit(X_train_smt, y_train_smt)
    y_pred_smt = rf_smt.predict(X_test)
    print(classification_report(y_test, y_pred_smt, target_names=['Legitimate', 'Fraud']))
    
except ImportError:
    print("\n⚠️ Install imbalanced-learn: pip install imbalanced-learn")
    print("Then re-run this cell.")

In [ ]:
# ============================================================
# Compare all methods
# ============================================================
print("\n" + "="*60)
print("COMPARISON: Fraud Detection F1-Scores")
print("="*60)

results = {
    'Baseline (no handling)': f1_score(y_test, y_pred_baseline, pos_label=1),
    'Class Weights': f1_score(y_test, y_pred_weighted, pos_label=1),
}

try:
    results['SMOTE'] = f1_score(y_test, y_pred_smote, pos_label=1)
    results['SMOTE + Tomek'] = f1_score(y_test, y_pred_smt, pos_label=1)
except:
    pass

for method, score in results.items():
    bar = '█' * int(score * 50)
    print(f"  {method:<25} F1={score:.4f} {bar}")

print("\n>>> Class weights and SMOTE significantly improve minority class detection!")

## 8. Exercises <a id='8-exercises'></a>

### Exercise 1: Convenience Sampling Bias
Take the first 1,000 records from the population (sorted by user_id). Compare the region distribution and fraud rate to the full population. Is it representative? What went wrong?

### Exercise 2: Temporal Sampling
Add a `timestamp` column to the population. Implement a temporal train/test split where the test set is the most recent 20% of data. Why is this better than random splitting for time-dependent data?

### Exercise 3: Custom Focal Loss
Implement a focal loss function in Python. Compare its behavior to standard cross-entropy loss for a highly imbalanced dataset.

---

## 9. Interview Preparation <a id='9-interview'></a>

### Q1: "You have a fraud detection dataset with 0.1% positives. How do you handle this?"

**Answer:**  
"I’d use a **multi-layered approach**:

1. **Evaluation metrics**: Never use accuracy. Use **Precision, Recall, F1, PR-AUC** (Precision-Recall AUC is better than ROC-AUC for severe imbalance)
2. **Stratified splitting**: Ensure train/test/validation all have the same 0.1% positive rate
3. **Algorithm-level**: Use `class_weight='balanced'` or focal loss to penalize missing fraud more heavily
4. **Data-level**: Apply **SMOTE** on training data only (never on test set!) to generate synthetic fraud examples
5. **Ensemble approaches**: BalancedRandomForest or EasyEnsemble for robust predictions
6. **Business context**: Set the decision threshold based on the cost ratio (cost of missing fraud vs cost of false alarm)"

---

### Q2: "What is stratified sampling and when would you use it?"

**Answer:**  
"Stratified sampling divides the population into **non-overlapping groups (strata)** and samples from each group proportionally.

I use it whenever:
- **Class imbalance exists** — ensures rare classes appear in both train and test sets
- **Important subgroups** — e.g., ensuring each country/demographic is represented
- **Train/test/val splits** — `sklearn.train_test_split(stratify=y)` preserves class proportions

Without stratification, a random split on 0.1% positive data could put ALL positive cases in the training set, leaving the test set with zero positive examples — making evaluation meaningless."

---

### Q3: "Explain the difference between oversampling and class weights. Which do you prefer?"

**Answer:**  
"**Oversampling** (like SMOTE) physically changes the dataset by adding synthetic minority samples. **Class weights** keep the original data but modify the loss function to penalize minority misclassification more.

Trade-offs:
- Oversampling **increases training time** (more data) but can capture complex decision boundaries
- Class weights are **computationally cheaper** and don’t risk overfitting to synthetic data
- SMOTE can **create noisy samples** near the class boundary

My preference:
- Start with **class weights** (simple, fast, no data augmentation risk)
- If that’s insufficient, try **SMOTE + Tomek Links** (oversample + clean boundary)
- Always validate on the **original, unmodified test set**"

---

### Q4: "What is reservoir sampling and when is it used in production?"

**Answer:**  
"Reservoir sampling is an **online algorithm** that maintains a random sample of fixed size `k` from a stream of unknown (potentially infinite) length, guaranteeing each item has equal probability `k/n` of being in the sample.

Production use cases:
1. **Monitoring**: Sampling 1% of model predictions for quality analysis without storing all predictions
2. **Log analysis**: Maintaining a representative sample of server logs for debugging
3. **A/B testing**: Randomly assigning streaming users to experiment groups
4. **Data collection**: Building training datasets from live traffic

The key advantage is **constant memory (O(k))** regardless of stream size."

---

### Q5: "SMOTE generated synthetic samples look good in training, but the model performs poorly in production. What went wrong?"

**Answer:**  
"Several common issues:

1. **SMOTE applied before train/test split** — synthetic samples leaked into the test set, giving overly optimistic evaluation
2. **Distribution mismatch** — SMOTE creates samples that don’t match the true production distribution. The model learns patterns that don’t exist in real data.
3. **Feature space not suitable** — SMOTE interpolates in feature space, which doesn’t make sense for all feature types (categorical, high-dimensional)
4. **Threshold not calibrated** — The decision threshold optimized on SMOTE-augmented data doesn’t translate to production

Prevention:
- Always apply SMOTE **only on training data**, evaluate on the original test set
- Use **probability calibration** after SMOTE
- Consider **class weights** as a simpler, more robust alternative"

---

## 🎓 Key Takeaways

1. **Sampling occurs everywhere** in ML — collection, labeling, splitting, training, monitoring
2. **Non-probability sampling** is quick but biased; use for prototyping only
3. **Stratified sampling** should be default for any classification problem
4. **Reservoir sampling** enables constant-memory sampling from streams
5. **Class imbalance** requires multi-layered handling: metrics, algorithm, data, and business context
6. **SMOTE** is powerful but must be applied only on training data

---

➡️ **Next Lesson**: [Lesson 4: Data Leakage](./lesson_04_data_leakage.ipynb) — The most insidious ML bug.